# 数据准备

In [41]:
import pandas as pd

df = pd.read_csv('data.csv')
print("原始数据：")
print(df)
print(df.info())

原始数据：
    id    age gender  score  height  weight status
0    1   23.0      M   85.0     175      70    已完成
1    2   21.0      F   90.0     160      55    已完成
2    3   22.0      M   78.0     180      80    进行中
3    4   20.0      F   95.0     158      50    已完成
4    5    NaN      M  120.0     170      68    已取消
5    6   25.0      F    NaN     165      60    已完成
6    7   19.0      M   70.0     172      65    已完成
7    8   24.0      F  105.0     162      54    进行中
8    9  130.0      M   82.0     178      75    已完成
9   10   23.0      F   85.0     160      55    已完成
10  11   22.0      M   88.0     300      75    已完成
11  12   -5.0      F   92.0     150      45    已完成
12  13   28.0      M  110.0     180     200    已完成
13  14   30.0      F   99.0     165      60    已完成
14  15   27.0      M  101.0     170      68    已完成
15  16   26.0      F    NaN     165      60    已完成
16  17   23.0      M   85.0     175      70    已完成
17  18   21.0      F    NaN     160      55    已完成
18  19   22.0      M   78

In [42]:
# 删除含缺失值的行
df_clean = df.dropna()

# # 用0填充所有缺失值
# df_filled = df.fillna(0)
#
# # 用每列均值填充
# df_mean = df.fillna(df.mean(numeric_only=True))
#
# # 针对某一列用中位数填充
# df['age'] = df['age'].fillna(df['age'].median())

# 查看前几行，确认格式和字段
print(df_clean.head())
# 查看字段类型和缺失情况
print(df_clean.info())

   id   age gender  score  height  weight status
0   1  23.0      M   85.0     175      70    已完成
1   2  21.0      F   90.0     160      55    已完成
2   3  22.0      M   78.0     180      80    进行中
3   4  20.0      F   95.0     158      50    已完成
6   7  19.0      M   70.0     172      65    已完成
<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 19
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      16 non-null     int64  
 1   age     16 non-null     float64
 2   gender  16 non-null     object 
 3   score   16 non-null     float64
 4   height  16 non-null     int64  
 5   weight  16 non-null     int64  
 6   status  16 non-null     object 
dtypes: float64(2), int64(3), object(2)
memory usage: 1.0+ KB
None


In [43]:
# 只保留年龄在0~120之间的记录
df_valid = df_clean[(df_clean['age'] >= 0) & (df_clean['age'] <= 120)]

# 只保留成绩在0~100之间的记录
df_valid = df_valid[(df_valid['score'] >= 0) & (df_valid['score'] <= 100)]

# 只保留状态为“已完成”的订单
df_valid = df_valid[df_valid['status'] == '已完成']

# 查看前几行，确认格式和字段
print(df_valid.head())


   id   age gender  score  height  weight status
0   1  23.0      M   85.0     175      70    已完成
1   2  21.0      F   90.0     160      55    已完成
3   4  20.0      F   95.0     158      50    已完成
6   7  19.0      M   70.0     172      65    已完成
9  10  23.0      F   85.0     160      55    已完成


In [45]:
# 作用：从数据集中分离出输入特征（X）。axis=1：按列删除（axis=0是按行删除）
X = df_valid.drop(['score', 'gender', 'status'], axis=1)
# 作用：提取目标变量（y）。选择 'score' 列作为预测目标
# 这是监督学习中的标签（label）
y = df_valid['score']
print("\n数值型特征X：")
print(X)
print("\n目标变量y：")
print(y)


数值型特征X：
    id   age  height  weight
0    1  23.0     175      70
1    2  21.0     160      55
3    4  20.0     158      50
6    7  19.0     172      65
9   10  23.0     160      55
10  11  22.0     300      75
13  14  30.0     165      60
16  17  23.0     175      70
19  20  20.0     158      50

目标变量y：
0     85.0
1     90.0
3     95.0
6     70.0
9     85.0
10    88.0
13    99.0
16    85.0
19    95.0
Name: score, dtype: float64


In [ ]:
from sklearn.feature_selection import VarianceThreshold

# 创建方差阈值选择器，设置阈值为0.0。
# threshold=0.0：删除方差为0的特征（即所有值都相同的列）
# 也可以设置其他阈值，如 threshold=0.1 删除方差小于0.1的特征
selector = VarianceThreshold(threshold=0.0)

# 对特征矩阵X进行方差阈值选择。
# fit_transform()：先拟合（计算每列方差），再转换（删除低方差列）
# 返回的是NumPy数组，列名信息丢失
X_var = selector.fit_transform(X)
# 获取被保留的列名。
# selector.get_support()：返回布尔数组，True表示保留的列
# indices=True：返回索引而不是布尔值
# X.columns[...]：根据索引获取对应的列名
selected_columns = X.columns[selector.get_support(indices=True)]
# 将NumPy数组转换回pandas DataFrame，并恢复列名。
X_var = pd.DataFrame(X_var, columns=selected_columns)
print("\n删除方差为0特征后的X：")
print(X_var)

In [51]:
# 作用：计算特征与目标变量的相关性矩阵。
# pd.concat([X_var, y], axis=1)：将特征矩阵X和目标变量y合并
# .corr()：计算皮尔逊相关系数矩阵
cor_matrix = pd.concat([X_var, y], axis=1).corr()
print("\n相关性矩阵：")
print(cor_matrix)

# 作用：找出与目标变量'score'相关性很低的特征。
# 获取目标变量'score'与其他特征的相关性,并筛选出小于0.3的特征
low_corr = cor_matrix['score'].abs() < 0.3
print("===========")
print(low_corr)
# 作用：获取需要删除的列名。
# 这是布尔索引，只保留值为 True 的行：
drop_cols = low_corr[low_corr].index
X_corr = X_var.drop(columns=drop_cols)

print("\n删除相关性低特征后的X：")
print(X_corr)


相关性矩阵：
              id       age    height    weight     score
id      1.000000  0.233823  0.069203 -0.031490 -0.674926
age     0.233823  1.000000 -0.016136  0.173211 -0.999220
height  0.069203 -0.016136  1.000000  0.681087  0.204757
weight -0.031490  0.173211  0.681087  1.000000  0.119523
score  -0.674926 -0.999220  0.204757  0.119523  1.000000
id        False
age       False
height     True
weight     True
score     False
Name: score, dtype: bool

删除相关性低特征后的X：
     id   age
0   1.0  23.0
1   2.0  21.0
2   4.0  20.0
3   7.0  19.0
4  10.0  23.0
5  11.0  22.0
6  14.0  30.0
7  17.0  23.0
8  20.0  20.0


In [52]:
from sklearn.feature_selection import SelectKBest, f_regression
# 创建特征选择器
# score_func=f_regression：使用F统计量作为评分函数
# k=min(3, X_corr.shape[1])：选择前k个特征，最多选3个，不超过现有特征数
# X_corr.shape[1]：当前特征矩阵的列数
selector = SelectKBest(score_func=f_regression, k=min(3, X_corr.shape[1]))
# 执行特征选择
# fit_transform()：先拟合（计算F统计量），再转换（选择特征）
# X_corr：输入特征矩阵、y：目标变量、 返回NumPy数组，只包含选中的特征
X_best = selector.fit_transform(X_corr, y)
# 获取被选中特征的列名
# selector.get_support()：返回布尔数组，True表示被选中的特征
# indices=True：返回索引而不是布尔值
# X_corr.columns[...]：根据索引获取对应的列名
best_columns = X_corr.columns[selector.get_support(indices=True)]
print("\nSelectKBest筛选后的特征：", list(best_columns))
print(pd.DataFrame(X_best, columns=best_columns))


SelectKBest筛选后的特征： ['id', 'age']
     id   age
0   1.0  23.0
1   2.0  21.0
2   4.0  20.0
3   7.0  19.0
4  10.0  23.0
5  11.0  22.0
6  14.0  30.0
7  17.0  23.0
8  20.0  20.0


In [53]:
# 基于特征选择后的数据进行预处理
print("原始特征数据（特征选择后）：")
print(X_best)
print("\n" + "="*50)







# 总结对比
print("数据预处理方法总结：")
print("1. Min-Max Scaling: 适合需要固定范围的数据，如[0,1]")
print("2. Z-Score标准化: 适合大多数机器学习算法，消除量纲影响")
print("3. 正态化: 适合需要计算距离的场景，如聚类、文本分类")
print("4. 二值化: 适合需要离散特征的算法，如朴素贝叶斯")

原始特征数据（特征选择后）：
[[ 1. 23.]
 [ 2. 21.]
 [ 4. 20.]
 [ 7. 19.]
 [10. 23.]
 [11. 22.]
 [14. 30.]
 [17. 23.]
 [20. 20.]]

1. Min-Max Scaling（0-1缩放）：
作用：将特征缩放到指定区间（0~1），消除不同量纲带来的影响
适用场景：需要将数据限制在特定范围，如神经网络输入
结果：
         id       age
0  0.000000  0.363636
1  0.052632  0.181818
2  0.157895  0.090909
3  0.315789  0.000000
4  0.473684  0.363636
5  0.526316  0.272727
6  0.684211  1.000000
7  0.842105  0.363636
8  1.000000  0.090909

2. Z-Score标准化：
作用：将特征转换为均值为0、方差为1的分布，适合大多数机器学习算法
适用场景：线性回归、SVM、神经网络等需要标准化特征的算法
结果：
         id       age
0 -1.363740  0.218218
1 -1.204342 -0.436436
2 -0.885545 -0.763763
3 -0.407351 -1.091089
4  0.070844  0.218218
5  0.230242 -0.109109
6  0.708436  2.509506
7  1.186631  0.218218
8  1.664825 -0.763763

3. 正态化（L2范数）：
作用：将每个样本缩放为单位范数，常用于文本特征或距离计算场景
适用场景：文本分类、聚类算法、需要计算样本间距离的场景
结果：
         id       age
0  0.043437  0.999056
1  0.094809  0.995495
2  0.196116  0.980581
3  0.345705  0.938343
4  0.398726  0.917070
5  0.447214  0.894427
6  0.422885  0.906183
7  0.594391  0.804

In [54]:
# 1. Min-Max Scaling（调整数据尺度）
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X_best)
print("1. Min-Max Scaling（0-1缩放）：")
print("作用：将特征缩放到指定区间（0~1），消除不同量纲带来的影响")
print("适用场景：需要将数据限制在特定范围，如神经网络输入")
print("结果：")
print(pd.DataFrame(X_scaled, columns=best_columns))
print("\n" + "="*50)


1. Min-Max Scaling（0-1缩放）：
作用：将特征缩放到指定区间（0~1），消除不同量纲带来的影响
适用场景：需要将数据限制在特定范围，如神经网络输入
结果：
         id       age
0  0.000000  0.363636
1  0.052632  0.181818
2  0.157895  0.090909
3  0.315789  0.000000
4  0.473684  0.363636
5  0.526316  0.272727
6  0.684211  1.000000
7  0.842105  0.363636
8  1.000000  0.090909



In [55]:
# 2. Z-Score标准化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X_best)
print("2. Z-Score标准化：")
print("作用：将特征转换为均值为0、方差为1的分布，适合大多数机器学习算法")
print("适用场景：线性回归、SVM、神经网络等需要标准化特征的算法")
print("结果：")
print(pd.DataFrame(X_standardized, columns=best_columns))
print("\n" + "="*50)


2. Z-Score标准化：
作用：将特征转换为均值为0、方差为1的分布，适合大多数机器学习算法
适用场景：线性回归、SVM、神经网络等需要标准化特征的算法
结果：
         id       age
0 -1.363740  0.218218
1 -1.204342 -0.436436
2 -0.885545 -0.763763
3 -0.407351 -1.091089
4  0.070844  0.218218
5  0.230242 -0.109109
6  0.708436  2.509506
7  1.186631  0.218218
8  1.664825 -0.763763



In [56]:
# 3. 正态化（Normalizer）
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(norm='l2')
X_normalized = normalizer.fit_transform(X_best)
print("3. 正态化（L2范数）：")
print("作用：将每个样本缩放为单位范数，常用于文本特征或距离计算场景")
print("适用场景：文本分类、聚类算法、需要计算样本间距离的场景")
print("结果：")
print(pd.DataFrame(X_normalized, columns=best_columns))
print("\n" + "="*50)

3. 正态化（L2范数）：
作用：将每个样本缩放为单位范数，常用于文本特征或距离计算场景
适用场景：文本分类、聚类算法、需要计算样本间距离的场景
结果：
         id       age
0  0.043437  0.999056
1  0.094809  0.995495
2  0.196116  0.980581
3  0.345705  0.938343
4  0.398726  0.917070
5  0.447214  0.894427
6  0.422885  0.906183
7  0.594391  0.804176
8  0.707107  0.707107



In [59]:
# 4. 二值化
from sklearn.preprocessing import Binarizer
# 创建一个二值化转换器，设置阈值为0.5。
# 含义：对于每个特征值，如果大于0.5则转换为1，否则为0。
binarizer = Binarizer(threshold=0.5)
X_binary = binarizer.fit_transform(X_normalized)
print("4. 二值化：")
print("作用：按阈值将特征转换为0或1，适合需要离散特征的模型")
print("适用场景：朴素贝叶斯、决策树等需要离散特征的算法")
print("结果：")
print(pd.DataFrame(X_binary, columns=best_columns))
print("\n" + "="*50)

4. 二值化：
作用：按阈值将特征转换为0或1，适合需要离散特征的模型
适用场景：朴素贝叶斯、决策树等需要离散特征的算法
结果：
    id  age
0  0.0  1.0
1  0.0  1.0
2  0.0  1.0
3  0.0  1.0
4  0.0  1.0
5  0.0  1.0
6  0.0  1.0
7  1.0  1.0
8  1.0  1.0



# 导出数据

In [60]:
import pandas as pd

# 假设 best_columns 是特征名
df_out = pd.DataFrame(X_best, columns=best_columns)
df_out['score'] = y.values  # 添加目标变量

# 查看结果
print(df_out.head())

     id   age  score
0   1.0  23.0   85.0
1   2.0  21.0   90.0
2   4.0  20.0   95.0
3   7.0  19.0   70.0
4  10.0  23.0   85.0


In [61]:
df_out.to_csv('processed.csv', index=False)
# index=False 表示不保存行索引到文件

In [62]:
df_loaded = pd.read_csv('processed.csv')
print(df_loaded.head())

     id   age  score
0   1.0  23.0   85.0
1   2.0  21.0   90.0
2   4.0  20.0   95.0
3   7.0  19.0   70.0
4  10.0  23.0   85.0
